In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('preprocessed.csv')

In [5]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

True
NVIDIA GeForce RTX 2050


In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached https://download.pytorch.org/whl/sympy-1.13.1-py3-none-any.whl (6.2 MB)
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ---------------------------------------- 536.2/536.2 kB 5.8 MB/s  0:00:00
   ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 GB 2.8 MB/s eta 0:15:08
   ---------------------------------------- 0.0/2.5 GB 2.8 MB/s eta 0:15:08
   ---------------------------------------- 0.0/2.5 GB 3.0 MB/s eta 0:13:51
   ---------------------------------------- 0.0/2.5 GB 2.4 MB/s eta 0:17:37
   ---------------------------------------- 0.0/2.5 GB 2.2 MB/s eta 0:19:24
   ---------------------------------------- 0.0/2.5 GB 2.1 MB/s eta 0:20:24
   --------------------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
"""
MLP with Attention layer for early-stage heart attack prediction
- Assumes data is a pandas DataFrame `df` with columns including target and features.
- Default target used here: `CVDCRHD4` (Ever diagnosed with angina/coronary heart disease) for early-stage prediction.
- Also supports `CVDINFR4` if you want to predict prior heart attack instead.

How to use:
1. Place your CSV (or other) and set `DATA_PATH` below.
2. Adjust `TARGET_COL` if needed.
3. Run: `python mlp_attention_heart_attack.py` (requires torch, sklearn, pandas, numpy)

Notes:
- This script performs basic cleaning of common survey-coded missing values (7,9,77,7777,9999...).
- It encodes categorical variables, scales numeric features, trains a small MLP with an attention block,
  and reports ROC AUC, accuracy, confusion matrix, and classification report.

"""

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix,recall_score
from sklearn.impute import SimpleImputer
from sklearn.utils import class_weight
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# ---------------------- User settings ----------------------
DATA_PATH = 'preprocessed.csv'  # replace with your path
TARGET_COL = 'CVDCRHD4'  # use 'CVDCRHD4' for early stage (angina/CHD) or 'CVDINFR4' for heart attack
RANDOM_STATE = 42
BATCH_SIZE = 256
EPOCHS = 40
LR = 1e-3
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# ---------------------- Utilities ----------------------
MISSING_CODES = {7,9,77,99,777,7777,999,9999}  # common survey missing/refused codes


def clean_survey_codes(df: pd.DataFrame):
    """Replace common survey missing/refused codes with NaN for numeric columns."""
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            df[col] = df[col].replace(list(MISSING_CODES), np.nan)
    return df


class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = X.astype(np.float32)
        self.y = y.astype(np.int64)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# ---------------------- Model with Attention ----------------------
class FeatureAttention(nn.Module):
    """Simple attention over feature-vector positions.
    Treats the feature vector as a sequence of length F with embedding dim 1.
    Projects features into a hidden dim, computes attention weights across feature positions,
    and produces a weighted representation.
    """
    def __init__(self, n_features, hidden_dim=64):
        super().__init__()
        self.key = nn.Linear(1, hidden_dim, bias=False)
        self.query = nn.Linear(1, hidden_dim, bias=False)
        self.value = nn.Linear(1, hidden_dim, bias=False)
        self.scale = hidden_dim ** 0.5

    def forward(self, x):
        # x: (batch, features)
        b, f = x.shape
        # reshape to (batch, features, 1)
        x_exp = x.view(b, f, 1)
        K = self.key(x_exp)    # (b, f, hidden)
        Q = self.query(x_exp)
        V = self.value(x_exp)
        # attention scores: (b, f, f)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / self.scale
        attn = torch.softmax(scores, dim=-1)
        # weighted sum: (b, f, hidden) -> then sum over source positions -> (b, hidden)
        out = torch.matmul(attn, V).sum(dim=1)
        return out  # (b, hidden)


class MLPWithAttention(nn.Module):
    def __init__(self, n_features, attn_hidden=64, mlp_hidden=[128, 64], dropout=0.2):
        super().__init__()
        self.attn = FeatureAttention(n_features, hidden_dim=attn_hidden)
        mlp_in = attn_hidden
        layers = []
        for h in mlp_hidden:
            layers.append(nn.Linear(mlp_in, h))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            mlp_in = h
        layers.append(nn.Linear(mlp_in, 2))  # binary classification -> 2 logits
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        # x: (batch, features)
        attn_out = self.attn(x)  # (batch, attn_hidden)
        logits = self.mlp(attn_out)
        return logits


# ---------------------- Training & Evaluation ----------------------

def train_epoch(model, loader, opt, loss_fn):
    model.train()
    total_loss = 0.0
    for xb, yb in loader:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        opt.zero_grad()
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        opt.step()
        total_loss += loss.item() * xb.size(0)
    return total_loss / len(loader.dataset)


def eval_model(model, loader, loss_fn):
    model.eval()
    total_loss = 0.0
    ys = []
    ys_pred = []
    ys_proba = []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(DEVICE)
            yb = yb.to(DEVICE)
            logits = model(xb)
            loss = loss_fn(logits, yb)
            total_loss += loss.item() * xb.size(0)
            probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            ys.extend(yb.cpu().numpy())
            ys_pred.extend(preds)
            ys_proba.extend(probs)
    return total_loss / len(loader.dataset), np.array(ys), np.array(ys_pred), np.array(ys_proba)


# ---------------------- Main pipeline ----------------------

def main():
    assert os.path.exists(DATA_PATH), f"DATA_PATH not found: {DATA_PATH}"
    df = pd.read_csv(DATA_PATH)
    print('Loaded', df.shape)

    df = clean_survey_codes(df)

    # choose target and drop rows with missing target
    df[TARGET_COL] = df[TARGET_COL].replace({1:1, 2:0})  # map: 1 -> positive, 2 -> negative
    df = df[df[TARGET_COL].isin([0,1])].copy()
    print('After dropping missing target:', df.shape)

    # basic feature selection: drop obvious meta cols if present
    drop_cols = [TARGET_COL]
    # You can expand drop_cols with survey ids like 'SEQN' etc.

    # Automatically determine numeric vs categorical
    feature_cols = [c for c in df.columns if c not in drop_cols]
    numeric_cols = [c for c in feature_cols if pd.api.types.is_numeric_dtype(df[c])]
    categorical_cols = [c for c in feature_cols if c not in numeric_cols]

    print(f'Numeric cols: {len(numeric_cols)}, Categorical cols: {len(categorical_cols)}')

    # Preprocessing pipelines
    numeric_pipeline = Pipeline([
        ('impute', SimpleImputer(strategy='median')),
        ('scale', StandardScaler()),
    ])
    categorical_pipeline = Pipeline([
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
    ])

    preprocessor = ColumnTransformer([
        ('num', numeric_pipeline, numeric_cols),
        ('cat', categorical_pipeline, categorical_cols),
    ])

    X = df[feature_cols]
    y = df[TARGET_COL].values

    # Fit transform
    X_proc = preprocessor.fit_transform(X)
    print('Processed feature matrix shape:', X_proc.shape)

    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_proc, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)

    # class weights to counter imbalance
    cw = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    cw_tensor = torch.tensor(cw, dtype=torch.float32).to(DEVICE)
    print('Class weights:', cw)

    # Datasets and loaders
    train_ds = TabularDataset(X_train, y_train)
    test_ds = TabularDataset(X_test, y_test)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

    n_features = X_train.shape[1]
    model = MLPWithAttention(n_features).to(DEVICE)

    # Weighted loss
    loss_fn = nn.CrossEntropyLoss(weight=cw_tensor)
    opt = torch.optim.Adam(model.parameters(), lr=LR)

    best_auc = 0.0
    for epoch in range(1, EPOCHS+1):
        train_loss = train_epoch(model, train_loader, opt, loss_fn)
        val_loss, y_true, y_pred, y_proba = eval_model(model, test_loader, loss_fn)
        try:
            auc = roc_auc_score(y_true, y_proba)
        except ValueError:
            auc = 0.0
        acc = accuracy_score(y_true, y_pred)
        print(f'Epoch {epoch:02d} | train_loss={train_loss:.4f} val_loss={val_loss:.4f} auc={auc:.4f} acc={acc:.4f}')
        if auc > best_auc:
            best_auc = auc
            torch.save(model.state_dict(), 'best_mlp_attention.pt')

    # Final evaluation
    model.load_state_dict(torch.load('best_mlp_attention.pt'))
    _, y_true, y_pred, y_proba = eval_model(model, test_loader, loss_fn)
    print('\nFinal evaluation on test set:')
    print('ROC AUC:', roc_auc_score(y_true, y_proba))
    print('Accuracy:', accuracy_score(y_true, y_pred))
    print('recall:', recall_score(y_true, y_pred))
    print('Confusion matrix:\n', confusion_matrix(y_true, y_pred))
    print('\nClassification report:\n', classification_report(y_true, y_pred))


if __name__ == '__main__':
    main()


Loaded (192169, 23)
After dropping missing target: (191079, 23)
Numeric cols: 22, Categorical cols: 0
Processed feature matrix shape: (191079, 22)
Class weights: [ 0.52601822 10.10864965]
Epoch 01 | train_loss=0.6160 val_loss=0.5954 auc=0.7296 acc=0.8271
Epoch 02 | train_loss=0.5960 val_loss=0.5892 auc=0.7324 acc=0.8312
Epoch 03 | train_loss=0.5908 val_loss=0.5886 auc=0.7278 acc=0.8306
Epoch 04 | train_loss=0.5898 val_loss=0.5923 auc=0.7383 acc=0.8842
Epoch 05 | train_loss=0.5912 val_loss=0.5863 auc=0.7373 acc=0.8511
Epoch 06 | train_loss=0.5890 val_loss=0.5868 auc=0.7368 acc=0.8635
Epoch 07 | train_loss=0.5888 val_loss=0.5869 auc=0.7379 acc=0.8660
Epoch 08 | train_loss=0.5903 val_loss=0.6056 auc=0.7389 acc=0.7939
Epoch 09 | train_loss=0.5892 val_loss=0.5875 auc=0.7362 acc=0.8337
Epoch 10 | train_loss=0.5890 val_loss=0.5861 auc=0.7364 acc=0.8503
Epoch 11 | train_loss=0.5901 val_loss=0.6007 auc=0.7358 acc=0.8026
Epoch 12 | train_loss=0.5888 val_loss=0.5867 auc=0.7360 acc=0.8655
Epoch 13

KeyboardInterrupt: 

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, recall_score, classification_report

# ===============================
# 🔧 Device Setup (GPU / CPU)
# ===============================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# ===============================
# 📊 Example Dataset (replace with your df)
# ===============================
df = pd.read_csv("preprocessed.csv")  # your actual dataset path
target_col = "CVDCRHD4"  # heart attack column (1=Yes, 2=No)

# Drop missing / unknown labels
df = df[df[target_col].isin([1, 2])]
df[target_col] = df[target_col].replace({1: 1, 2: 0})

# Split features and target
X = df.drop(columns=[target_col])
y = df[target_col]

# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = X.select_dtypes(exclude=["int64", "float64"]).columns

# ===============================
# 🧹 Preprocessing
# ===============================
numeric_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale', StandardScaler())
])
categorical_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, numeric_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

X_processed = preprocessor.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# ===============================
# 🧠 Model Definition
# ===============================
class AttentionLayer(nn.Module):
    def __init__(self, input_dim):
        super(AttentionLayer, self).__init__()
        self.attn = nn.Sequential(
            nn.Linear(input_dim, input_dim),
            nn.Tanh(),
            nn.Linear(input_dim, 1)
        )

    def forward(self, x):
        attn_weights = torch.softmax(self.attn(x), dim=1)
        context = torch.sum(attn_weights * x, dim=1)
        return context

class MLPWithAttention(nn.Module):
    def __init__(self, input_dim):
        super(MLPWithAttention, self).__init__()
        self.attn = AttentionLayer(input_dim)
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        context = self.attn(x)
        out = self.net(x)
        return out

model = MLPWithAttention(X_train.shape[1]).to(device)

# ===============================
# ⚙️ Training Setup
# ===============================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
epochs = 40

# ===============================
# 🚀 Training Loop
# ===============================
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

# ===============================
# 🧾 Evaluation
# ===============================
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        preds = model(xb)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(torch.argmax(preds, dim=1).cpu().numpy())

acc = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
print(f"\n✅ Accuracy: {acc*100:.2f}%")
print(f"📈 Recall: {recall*100:.2f}%")
print("\n📋 Classification Report:\n", classification_report(y_true, y_pred))


Using device: cuda
Epoch 1/40, Loss: 0.1745
Epoch 2/40, Loss: 0.1453
Epoch 3/40, Loss: 0.1426
Epoch 4/40, Loss: 0.1417
Epoch 5/40, Loss: 0.1413
Epoch 6/40, Loss: 0.1405
Epoch 7/40, Loss: 0.1400
Epoch 8/40, Loss: 0.1401
Epoch 9/40, Loss: 0.1399
Epoch 10/40, Loss: 0.1392
Epoch 11/40, Loss: 0.1393
Epoch 12/40, Loss: 0.1394
Epoch 13/40, Loss: 0.1387
Epoch 14/40, Loss: 0.1386
Epoch 15/40, Loss: 0.1386
Epoch 16/40, Loss: 0.1387
Epoch 17/40, Loss: 0.1382
Epoch 18/40, Loss: 0.1382
Epoch 19/40, Loss: 0.1380
Epoch 20/40, Loss: 0.1380
Epoch 21/40, Loss: 0.1376
Epoch 22/40, Loss: 0.1376
Epoch 23/40, Loss: 0.1377
Epoch 24/40, Loss: 0.1375
Epoch 25/40, Loss: 0.1373
Epoch 26/40, Loss: 0.1371
Epoch 27/40, Loss: 0.1370
Epoch 28/40, Loss: 0.1371
Epoch 29/40, Loss: 0.1372
Epoch 30/40, Loss: 0.1371
Epoch 31/40, Loss: 0.1369
Epoch 32/40, Loss: 0.1368
Epoch 33/40, Loss: 0.1367
Epoch 34/40, Loss: 0.1366
Epoch 35/40, Loss: 0.1366
Epoch 36/40, Loss: 0.1364
Epoch 37/40, Loss: 0.1362
Epoch 38/40, Loss: 0.1359
Ep